In [18]:
# Following is the tutorial on how to create and deploy a Matching Engine index which will store your data & embeddings
# After you deploy the index, you will be able to query it for semantically similar data, to, for example, get answers for a
# question, like it was shown on the ChatDKG Office Hours #5

In [4]:
from google.colab import auth
auth.authenticate_user() # Authenticate first so you can use the Google Cloud commands

PROJECT_ID = "ot-demo"

In [5]:
!gcloud config set project "ot-demo"

Updated property [core/project].


In [6]:
!pip install google-cloud-aiplatform==1.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1


In [1]:
from google.cloud import aiplatform
from google.protobuf import struct_pb2
import time

In [2]:
aiplatform.init(project="ot-demo", location="us-central1")

In [5]:
REGION = "us-central1"
ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
print(PROJECT_NUMBER)

185434836498


In [9]:
NETWORK_NAME = "ot-vpc-network" # This name will be visible in the interface
PEERING_RANGE_NAME = "ot-range" # This name will be visible in the interface

# Below is the setup to create a VPC network for your Google Vertex AI project, which is needed to deploy the index

In [10]:
! gcloud compute networks create {NETWORK_NAME} --bgp-routing-mode=regional

ERROR: (gcloud.compute.networks.create) Could not fetch resource:
 - The resource 'projects/vertex-ka/regions/us-central1/subnetworks/ot-matching-engine-vpc-network' already exists



In [8]:
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-icmp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow icmp
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-internal --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-rdp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:3389
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-ssh --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:22

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/vertex-ka/global/firewalls/ot-matching-engine-vpc-network-allow-icmp' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/vertex-ka/global/firewalls/ot-matching-engine-vpc-network-allow-internal' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/vertex-ka/global/firewalls/ot-matching-engine-vpc-network-allow-rdp' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/vertex-ka/global/firewalls/ot-matching-engine-vpc-network-allow-ssh' already exists



In [11]:
! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={NETWORK_NAME} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range for uCAIP Haystack."

ERROR: (gcloud.compute.addresses.create) Could not fetch resource:
 - The resource 'projects/vertex-ka/global/addresses/ot-matching-engine-range' already exists



In [12]:
!gcloud services enable servicenetworking.googleapis.com --project="ot-demo"

In [13]:
! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={NETWORK_NAME} --ranges={PEERING_RANGE_NAME}

Operation "operations/pssn.p24-185434836498-ddca46db-0557-43d3-8e96-c207944c5e55" finished successfully.


In [19]:
# Create a Google Cloud bucket: https://console.cloud.google.com/storage/browser in the same!!! region as your project
# Copy your data.json from the notebook to the bucket, this can also be done from the interface
# Data.json as of writing of this comment should be in the following format:
# Every line should contain a valid JSON object with the properties 'id' with a string value and 'embedding' with an array value
! gsutil cp data.json gs://ot-demox/matching-engine/data/text/data.json

BucketNotFoundException: 404 gs://ot-demox bucket does not exist.


In [15]:
index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name="ot_index_display_name",
    contents_delta_uri="gs://ot-demo/matching-engine/data/text/", # path to the data on the bucket
    dimensions=768, # dimensions of the vector, make sure this matches the embedding model dimension you use
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE", # distance function your model uses, check this on huggingfaces for example
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    description="description here",
    labels={"label_name": "label_value"},
)

Creating MatchingEngineIndex


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex


Create MatchingEngineIndex backing LRO: projects/185434836498/locations/us-central1/indexes/620854633786507264/operations/5423429820849586176


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/185434836498/locations/us-central1/indexes/620854633786507264/operations/5423429820849586176


KeyboardInterrupt: ignored

In [16]:
# check whether your index was created via this command or the interface
!gcloud ai indexes list \
  --project="ot-demo" \
  --region="us-central1"

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
---
createTime: '2023-08-08T14:44:15.555876Z'
deployedIndexes:
- deployedIndexId: ot_deployed_index_v3
  indexEndpoint: projects/185434836498/locations/us-central1/indexEndpoints/195264468999995392
- deployedIndexId: ot_demo_index_deployed
  indexEndpoint: projects/185434836498/locations/us-central1/indexEndpoints/2801722753340669952
- deployedIndexId: ot_demo_deployed_index
  indexEndpoint: projects/185434836498/locations/us-central1/indexEndpoints/2801722753340669952
- deployedIndexId: similar_text_index_deployed
  indexEndpoint: projects/185434836498/locations/us-central1/indexEndpoints/2801722753340669952
- deployedIndexId: ot_similar_text_index_deployed
  indexEndpoint: projects/185434836498/locations/us-central1/indexEndpoints/2801722753340669952
- deployedIndexId: ot_deployed_index_v2
  indexEndpoint: projects/185434836498/locations/us-central1/indexEndpoints/6185051973402755072
description: Glove 100 ANN index
disp